## **P4 (Містобудування, ЖКГ, побутове, торговельне обслуговування, транспорт і зв’язок): 3/6**

In [1]:
import numpy as np
import pandas as pd
from custom_func import *

### **p4_02**
#### наявність генеральних планів у містах та селищах міського типу
### **p4_03**
#### наявність генеральних планів у селах

In [2]:
_dict_ids = population.to_dict()["p4_region_id"]
_dict_ids_reversed = {v:k for k, v in _dict_ids.items()}

df_P04_005 = pd.read_excel(f"{INPUTS_PATH}/P4/P04_005.xlsx", dtype={"КОАТУУ населеного пункту": str})
df_P04_005 = df_P04_005.loc[df_P04_005["region_id"].ne("26_kyiv")].copy()

urban = ["місто", "смт"]
rural = ["село",  "селище"]

# додаю так, щоб мати змогу по-іншому визначити urban / rural, не змінюючи оригінальні дані
df_P04_005["param_type"] = np.select(
    condlist=[
        df_P04_005["Тип населеного пункту (село, селище міського типу, місто)"].isin(urban),
        df_P04_005["Тип населеного пункту (село, селище міського типу, місто)"].isin(rural)
    ],
    choicelist=["p4_02_raw", "p4_03_raw"],
    default= np.nan
)

df_P04_005["plan"] = np.where(df_P04_005["Наявність затвердженого Генерального плану населеного пункту (так/ні)"].eq("так"), 1,0)
df_P04_005_fin = df_P04_005.groupby(["region_id", "param_type"], as_index=False)["plan"].mean()
df_P04_005_fin = df_P04_005_fin.pivot(index='region_id', columns='param_type', values='plan').reset_index()
df_P04_005_fin["region"] = df_P04_005_fin["region_id"].map(_dict_ids_reversed)
df_P04_005_fin = df_P04_005_fin.drop("region_id", 1)

normalize_parameter(df_P04_005_fin,'p4_02_raw','p4_02',min_bound=0,max_bound=1)
normalize_parameter(df_P04_005_fin,'p4_03_raw','p4_03',min_bound=0,max_bound=1)

Емпіричні границі: (0, 1)
Коефіцієнти нормалізації: (1.0, -0.0)
Параметр p4_02 нормалізовано і додано до таблиці

Емпіричні границі: (0, 1)
Коефіцієнти нормалізації: (1.0, -0.0)
Параметр p4_03 нормалізовано і додано до таблиці



### **p4_05**
#### протяжність автомобільних доріг загального користування місцевого значення, на яких проведено капітальний ремонт, реконструкцію, відносно загальної довжини доріг

In [3]:
df_P04_006 = pd.read_excel(f"{INPUTS_PATH}/P4/P04_006.xlsx")
roads = pd.read_csv(f"{INPUTS_PATH}/P99/all_roads.csv").rename(columns={"Область": "region"})

In [4]:
df_P04_006 = df_P04_006.loc[pd.to_datetime(df_P04_006["Дата початку ремонтних робіт"]) < "2020-07-01"]

df_P04_006_fin = pd.concat([
    roads.groupby("region")["Довжина (протяжність) дороги (км)"].sum().rename('roads_all'),
    df_P04_006.groupby('region')['Довжина (протяжність) відремонтованої ділянки (км)'].sum().rename('roads_repaired'),
    df_P04_006.groupby('region')['Загальна вартість ремонту (грн)'].sum().rename('price')
], axis=1)

df_P04_006_fin['price_km'] = df_P04_006_fin['price']/df_P04_006_fin['roads_repaired']
df_P04_006_fin['p4_05_raw'] = df_P04_006_fin['roads_repaired']/df_P04_006_fin['roads_all']
df_P04_006_fin = df_P04_006_fin.reset_index().rename(columns={"index": "region"})
normalize_parameter(df_P04_006_fin,'p4_05_raw','p4_05',min_bound=0)

Емпіричні границі: (0, 0.020276599459073542)
Коефіцієнти нормалізації: (49.31793430246567, -0.0)
Параметр p4_05 нормалізовано і додано до таблиці



### **p4_06**
#### рівень використання коштів дорожнього фонду, передбачених для будівництва, реконструкції, ремонту та утримання автомобільних доріг загального користування місцевого значення, вулиць і доріг комунальної власності у населених пунктах

In [5]:
df_KMU["p4_06_raw"] = df_KMU["Дор фонд_видатки"] / df_KMU["Дор фонд_передбачено"]
normalize_parameter(df_KMU, "p4_06_raw", "p4_06")

Емпіричні границі: (0.022008531339642, 0.6319770035765523)
Коефіцієнти нормалізації: (1.6394289959491584, -0.03608142443646487)
Параметр p4_06 нормалізовано і додано до таблиці



### **p4_07**
#### середній рівень будівельної готовності об'єктів Великого будівництва (за касовими видатками)
### **p4_08**
#### середній рівень освоєння коштів, виділених на об'єкти Великого будівництва з усіх джерел

In [6]:
df_P04_009 = pd.read_excel(f"{INPUTS_PATH}/P4/P04_009.xlsx")

In [7]:
df_P04_009["p4_08_raw"] = df_P04_009["Видатки_усього"] / df_P04_009["Профінансовано_усього"]
df_P04_009_fin = (
    df_P04_009
    .loc[df_P04_009["Область"].ne("м. Київ")]
    .rename(columns={"Готовність_від факту": "p4_07_raw", "Область": "region"})
    .groupby("region", as_index=False)[["p4_07_raw", "p4_08_raw"]].mean()
)

normalize_parameter(df_P04_009_fin, "p4_07_raw", "p4_07")
normalize_parameter(df_P04_009_fin, "p4_08_raw", "p4_08")

Емпіричні границі: (0.1948777777777778, 0.555)
Коефіцієнти нормалізації: (2.776835025145784, -0.5411434389559101)
Параметр p4_07 нормалізовано і додано до таблиці

Емпіричні границі: (0.2562015732871442, 0.9309094497439903)
Коефіцієнти нормалізації: (1.4821229081412086, -0.3797222208706952)
Параметр p4_08 нормалізовано і додано до таблиці



### **Група 1:** p4_02-p4_03, генплани
### **Група 2:** p4_05-p4_06, дороги
### **Група 3:** p4_07-p4_08, велике будівництво

In [8]:
dict_weights = {
    'p4_02':1,
    'p4_03':1,
    'p4_05':1,
    'p4_06':1,
    'p4_07':1,
    'p4_08':1
}

sources = [
    df_P04_005_fin, 
    df_P04_006_fin,
    df_KMU,
    df_P04_009_fin
]
save_data(sources,regions,dict_weights,'P4')

Дані за галузевим параметром P4 збережено
               region     p4_02     p4_03     p4_05     p4_06     p4_07  \
0           Волинська  1.000000  0.877979  0.526775  0.128628  0.633896   
1           Вінницька  0.957447  0.797529  0.318542  0.480516  0.459674   
2    Дніпропетровська  0.742424  0.540070  0.948457  0.627844  0.511079   
3            Донецька  0.561798  0.433681  0.276283  0.895208  0.973575   
4         Житомирська  0.295775  0.297111  0.377780  0.289077  0.860233   
5        Закарпатська  1.000000  0.929188  0.422385  0.102066  0.723385   
6          Запорізька  0.873418  0.587356  0.362038  0.317821  0.515011   
7            Київська  0.963636  0.827709  0.434851  0.531625  0.576070   
8   Івано-Франківська  1.000000  1.000000  1.000000  0.200172  0.606152   
9      Кіровоградська  0.878788  0.687966  0.204456  0.299042  0.205699   
10          Луганська  1.000000  0.706931  0.653571  0.338171  0.929191   
11          Львівська  0.830986  0.547018  0.341205  1.000